In [39]:
from oven_time import processing
import pandas as pd

tec = "NUCLEAR"
at_time = "2025-11-27 12:05:00+01:00"
window = 24

In [40]:
# --- Normaliser tec ---
single = False
if isinstance(tec, str):
    tec = [tec]
    single = True

tec

['NUCLEAR']

In [41]:
# --- Charger les données ---
data = processing.init_data()
data.head()

,load,RENEWABLE,NUCLEAR,STORAGE,GAS,COAL,OTHER_DISP
time,,,,,,,
2025-11-20 14:00:00+00:00,65935.38,14004.83,48911.18,2653.41,6963.30,516.03,802.30
2025-11-20 14:15:00+00:00,65340.37,13212.79,48852.95,3075.78,7029.83,497.17,805.96
2025-11-20 14:30:00+00:00,65282.99,12583.49,48903.23,3375.07,6873.24,524.11,807.05
2025-11-20 14:45:00+00:00,64970.76,11907.19,48917.29,3548.13,6873.06,525.00,812.87
2025-11-20 15:00:00+00:00,65127.50,11217.56,48918.43,4532.23,7021.06,523.09,796.25


In [42]:
# Pas vraiment nécessaire

# --- S'assurer d'un DatetimeIndex tz-aware ---
if not isinstance(data.index, pd.DatetimeIndex):
    data.index = pd.to_datetime(data.index)
    print("Needs to convert")

if data.index.tz is None:
    # ENTSO-E renvoie de l’Europe/Paris : on suppose cela
    data.index = data.index.tz_localize("Europe/Paris")
    print("Specifies the time zone.")

In [43]:
# --- Convertir at_time ---
if at_time is None:
    target_time = data.index.max()
else:
    target_time = pd.Timestamp(at_time)

    if target_time.tzinfo is None:
        target_time = target_time.tz_localize(data.index.tz)
    else:
        target_time = target_time.tz_convert(data.index.tz)

    target_time = target_time.floor("15min")

print("Target time set at "+str(target_time)+".")

Target time set at 2025-11-27 11:00:00+00:00.


In [44]:
# --- Extraire la fenêtre ---
idx_target = data.index.get_loc(target_time)
start_idx = idx_target - window + 1

# Vérifier que la fenêtre est valide
if start_idx < 0:
    first_available = data.index.min()
    raise ValueError(
        f"Window of size {window} is {-start_idx} too large "
        f"(first timestamp = {first_available})."
    )

# Si OK, on extrait
window_df = data.iloc[start_idx : idx_target + 1]

window_df

,load,RENEWABLE,NUCLEAR,STORAGE,GAS,COAL,OTHER_DISP
time,,,,,,,
2025-11-27 05:15:00+00:00,65280.77,12633.94,51977.48,3311.50,3255.35,408.24,855.51
2025-11-27 05:30:00+00:00,66406.40,12700.63,51904.11,3967.23,3385.12,433.99,856.59
2025-11-27 05:45:00+00:00,68220.91,12848.10,51926.11,4799.96,3981.79,516.92,831.98
2025-11-27 06:00:00+00:00,69686.87,13510.80,51741.72,4919.01,4464.54,484.58,826.01
2025-11-27 06:15:00+00:00,71195.41,13717.01,51733.89,5866.03,4581.74,450.82,815.08
2025-11-27 06:30:00+00:00,71696.09,13922.30,51698.52,6370.18,4721.12,448.77,1003.95
2025-11-27 06:45:00+00:00,72355.51,14033.28,51580.96,6389.06,4822.27,446.84,1496.72
2025-11-27 07:00:00+00:00,72398.28,14281.25,51700.59,6351.67,4994.07,449.64,1519.98
2025-11-27 07:15:00+00:00,72478.57,14477.61,51754.11,6361.01,5026.97,443.17,1512.50


In [46]:
# --- Calculs ---
result = {}
for t in tec:
    series = window_df[t].dropna()

    if series.empty:
        result[t] = float("nan")
        continue

    mn = series.min()
    mx = series.max()

    if mx == mn:
        result[t] = float("nan")
    else:
        result[t] = (series.iloc[-1] - mn) / (mx - mn)

result

{'NUCLEAR': np.float64(0.2548208070736456)}

In [21]:
from oven_time import decision

decision.cycle_whereat("NUCLEAR", at_time="2025-12-01 02:00", window=6, mode="zero_to_max")

np.float64(0.9901928830371776)

In [8]:
from oven_time import decision
import pandas as pd

decision.call_minTOmax(["NUCLEAR","STORAGE","GAS","COAL"], window=24, at_time="2025-11-30 14:00")


Time: 2025-11-30 14:00:00+00:00


{'NUCLEAR': np.float64(0.18342990999876435),
 'STORAGE': np.float64(0.12424392970253988),
 'GAS': np.float64(0.7730347684723899),
 'COAL': nan}

In [3]:
print(pd.Timestamp("2025-11-27 12:00", tz="Europe/Paris").tz_convert(tz='UTC'))

2025-11-27 11:00:00+00:00


In [9]:
from oven_time import decision

decision.diagnostic()

Gas-fired generation is at 4% of its range over the past 24h.
Flexible generation (hydro, storage) is at 11% of its range over the past 24h.
Flexible generation (hydro, storage) is at 29% of its generation capacity over the past 24h.
Nuclear is at 99.6% of the recently available capacity.

LOOKING GOOD! Gas generation in a very low phase (even though nuclear is at max)


np.float64(140.51138003651096)

In [ ]:
from oven_time import decision

decision.cycle_whereat("GAS", at_time="2025-11-30 13:30", window=24, mode="zero_to_max")

np.float64(1.0)